# Run stock-price prediction model

## Notebook ([`papermill`](https://papermill.readthedocs.io/en/latest/index.html)) parameters

In [1]:
run_time = None
stop_minutes = 10
dir = None
earliest_data_cutoff = '2019-04-01'
ticker = 'AAPL'
batch_size = 100000
max_epochs = 10000

## Date/Time helpers

In [2]:
from sys import executable as python
from datetime import datetime as dt, date, time, timedelta as Δ
from dateutil.parser import parse

strptime = dt.strptime
now = dt.now()
today = now.date()

if run_time:
    run_time = parse(run_time)
else:
    run_time = dt.now()

run_date = run_time.date()

if stop_minutes:
    stop_at = run_time + Δ(minutes=stop_minutes)
else:
    stop_at = None

earliest_data_cutoff = parse(earliest_data_cutoff)
earliest_data_cutoff_date = earliest_data_cutoff.date()
    
print('Running at %s (stopping within %dmins)' % (run_time.strftime('%Y-%m-%d %H:%M:%S'), stop_minutes))

Running at 2019-07-01 10:00:00 (stopping within 10mins)


## Initialize `data`, `models` directories

In [3]:
from pathlib import Path
if not dir:
    dir = Path.cwd()

data_dir = dir / 'data'
data_dir.mkdir(parents=True, exist_ok=True)
models_dir = dir / 'models'

## Read IEX API credentials
Search for a secret key in:
- `$IEX_SECRET_KEY` env var
- `iex.secret_key` in a config file located at:
  - `$CONFIG_PATH` if set
  - `~/.config/iex.ini` otherwise

In [4]:
from os import environ as env

api = 'https://cloud.iexapis.com'

SECRET_KEY_VAR = 'IEX_SECRET_KEY'
CONFIG_PATH_VAR = 'CONFIG_PATH'

if SECRET_KEY_VAR in env:
    secret_key = env[SECRET_KEY_VAR]
else:
    if CONFIG_PATH_VAR in env:
        config_path = Path(env[CONFIG_PATH_VAR])
    else:
        config_path = Path.home() / '.config' / 'iex.ini'

    if not config_path.exists():
        raise Exception("No %s env var, and config path %s doesn't exist" % (SECRET_KEY_VAR, config_path))

    from configparser import ConfigParser
    config = ConfigParser()
    config.read(str(config_path))
    iex_config = config['iex']
    secret_key = iex_config['secret_key']

## Initialize `dates` and `minutes` that trading occurs during

In [5]:
start_minute = time(9, 30)
end_minute = time(16, 0)
def get_minutes():
    minute = start_minute
    while minute < end_minute:
        yield minute
        hr = minute.hour
        min = minute.minute
        min += 1
        if min == 60:
            min = 0
            hr += 1
        minute = time(hr, min)

minutes = list(get_minutes())
num_minutes = len(minutes); num_minutes

def get_dates(start_date, end_date, step=1):
    date = start_date
    while date != end_date:
        # only emit weekdays
        if date.weekday() <= 4:
            yield date
        date += Δ(days=step)

start_date = earliest_data_cutoff_date
end_date = run_date + Δ(days=1)
dates = list(get_dates(start_date, end_date))
num_dates = len(dates)

print(
    'Fetching/Processing data from %d days ([%s,%s)), %d minutes per day' % (
        num_dates, 
        start_date.strftime('%Y-%m-%d'),
        end_date.strftime('%Y-%m-%d'),
        num_minutes,
    )
)

Fetching/Processing data from 66 days ([2019-04-01,2019-07-02)), 390 minutes per day


In [6]:
def get_ticker_dir(ticker):
    return data_dir / ticker

def get_ticker_date_path(ticker, date):
    return get_ticker_dir(ticker) / date.strftime('%Y%m%d')

In [7]:
!{python} -m pip install -Uq requests
from requests import get as GET

In [8]:
import json

def fetch(date, ticker, refetch_partial=False):
    date_str = date.strftime('%Y%m%d')
    out_path = get_ticker_date_path(ticker, date)
    refetch = False
    prev_data = None
    if out_path.exists():
        if refetch_partial:
            with out_path.open('r') as f:
                prev_data = json.load(f)
                if len(prev_data) < num_minutes:
                    refetch = True
                    print(
                        'Re-fetching data for %s from %s (found %d per-minute quotes instead of %d)' % (
                            ticker, 
                            date_str, 
                            len(prev_data), 
                            num_minutes
                        )
                    )
                else:
                    return True
        else:
            return True
    else:
        print('Fetching data for %s from %s' % (ticker, date_str))

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    data = json.loads(resp.content)
    if prev_data is None or len(data) > len(prev_data):
        with out_path.open('wb') as f:
            f.write(resp.content)

        if refetch:
            print('Re-fetch found data for %s %s' % (date_str, ticker))
        return True
    elif len(data) < len(prev_data):
        raise Exception('Found %d data, less than previous amount %d' % (len(data), len(prev_data)))
    else:
        print('Re-fetched %s, found same %d data' % (out_path, len(data)))

    return False

In [9]:
!{python} -m pip install -Uq joblib
from joblib import Parallel, delayed

Fetch quotes between a start point in the past (IEX seems to serve 6-7mos of historic data) and today

In [10]:
%%time

N = 32  # fetch parallelism
refetch_empty = False

Parallel(n_jobs=N)(
    delayed(fetch)(date, ticker, refetch_partial=refetch_empty or date == today)
    for date in dates 
); None

CPU times: user 399 ms, sys: 214 ms, total: 613 ms
Wall time: 7.22 s


In [11]:
!{python} -m pip install -Uq pandas matplotlib numpy scipy
import numpy as np
from numpy import \
    array, \
    nan, isnan as na, \
    zeros, \
    count_nonzero as cnz, \
    mean, std, \
    unique, \
    logical_and as l_and, \
    logical_or as l_or, \
    exp, log, sqrt
from numpy.random import shuffle, permutation
from pandas import concat, DataFrame as DF, read_csv, read_json
import pandas as pd
from scipy.stats import describe

These are the columns that we receive from IEX, describing trades that took place there:

In [12]:
features = [ 'open', 'close', 'high', 'low', 'average', 'volume', 'notional', 'numberOfTrades' ]

In [13]:
def load_ticker_date_df(date, ticker, latest_data_datetime=None):
    date_str = date.strftime('%Y%m%d')
    out_path = get_ticker_date_path(ticker, date)
    if not out_path.exists():
        return None
    df = read_json(out_path)
    if df.empty:
        return None
    # Convert "date", "minute" columns into a single "datetime" column
    df['datetime'] = df['date'].apply(lambda d: d.strftime('%Y-%m-%d')) + ' ' + df['minute']
    df['datetime'] = df['datetime'].apply(lambda s: strptime(s, '%Y-%m-%d %H:%M'))
    df.drop(columns=['date', 'minute'])
    if latest_data_datetime:
        df = df.loc[df.datetime <= latest_data_datetime]
    
    # Drop other columns ("market"-data columns, which are updated on a 15-min delay)
    df = df[['datetime'] + features]

    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)

    return df

In [14]:
def load_ticker_df(ticker, N=None, limit=None):
    if limit is None:
        ds = dates
    elif type(limit) == int:
        ds = dates[:limit]
    elif type(limit) == date:
        ds = [ date for date in dates if date < limit ]        
    elif type(limit) == dt:
        ds = [ date for date in dates if date < limit.date() ]
    else:
        raise Exception('Unrecognized limit: %s' % limit)

    if N is None:
        df = concat([ load_ticker_date_df(date, ticker, latest_data_datetime=run_time) for date in ds ])
    else:
        df = concat(Parallel(n_jobs=N)( delayed(load_ticker_date_df)(date, ticker) for date in ds ))
    
    for col in features:
        # "quantity" features (volume, numberOfTrades, notional) are sometimes -1 instead of NaN
        df[col] = df[col].apply(lambda n: nan if n < 0 else n)

    return df

In [15]:
%%time
aapl = load_ticker_df('AAPL'); aapl

CPU times: user 1.91 s, sys: 40.2 ms, total: 1.95 s
Wall time: 2.01 s


,open,close,high,low,average,volume,notional,numberOfTrades
datetime,,,,,,,,
2019-04-01 09:30:00,191.645,190.65,191.645,190.600,191.189,4320,825935.940,44
2019-04-01 09:31:00,190.700,190.98,190.980,190.640,190.761,3246,619210.510,32
2019-04-01 09:32:00,191.060,190.93,191.090,190.780,190.951,2253,430211.740,30
2019-04-01 09:33:00,190.980,190.83,191.010,190.760,190.946,2241,427911.290,27
2019-04-01 09:34:00,190.760,190.70,190.760,190.600,190.666,1069,203822.465,12
...,...,...,...,...,...,...,...,...
2019-07-01 09:56:00,203.280,203.24,203.315,203.225,203.275,4513,917380.920,43
2019-07-01 09:57:00,203.210,203.38,203.405,203.210,203.348,1804,366839.520,20
2019-07-01 09:58:00,203.390,203.59,203.685,203.370,203.518,6396,1301698.610,69


## Fill missing data
Minutes where no trades were recorded can be assigned prices based on previous minutes' closing prices

We don't attempt to cross day boundaries, as the moves overnight are often much larger than minute-to-minute moves during the day, and skew the data.

Make a series representing minutes that are the first of their day (i.e. 9:30am)

In [16]:
prev_datetime = aapl.index.to_series().shift(1)
datetime = aapl.index.to_series()

def to_date(dt):
    return dt.date()

day_start = datetime.apply(to_date) != prev_datetime.apply(to_date)

Populate each minute with the previous minute's closing price

Assume each day's initial opening is the same as an imagined closing price from a minute prior (when trading wasn't open).

In [17]:
prev_close = aapl.close.shift(1)
prev_close[day_start] = aapl.open
aapl['prev_close'] = prev_close

Track minutes where no trades occurred:

In [18]:
nan_idxs = aapl[features].isna().any(axis=1); nan_idxs
nans = aapl[nan_idxs]
nans

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,193.000
2019-04-05 13:36:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.540
2019-04-05 14:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.795
2019-04-15 13:33:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,198.910
2019-04-16 13:39:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,199.725
2019-04-18 14:27:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,203.820
2019-04-18 14:44:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,203.920
2019-04-24 13:47:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,207.570
2019-04-30 12:25:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,200.830


Spot-check: consecutive pairs of minutes where no trades occurred

In [19]:
nan_pairs = l_and(nan_idxs, nan_idxs.shift(1).fillna(False))
concat([ aapl[nan_pairs], aapl[nan_pairs.shift(-1).fillna(False)] ]).sort_index()

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,


Group by date, and forward-fill "previous closing" price.

For minutes when no trades occurred, we will treat the last valid closing price as both the opening and closing price. However, we don't persist these across day boundaries (this results in apparent large jumps at opening that skew training).

In [20]:
grouped = aapl.groupby(to_date)
aapl.prev_close = grouped.prev_close.apply(lambda s: s.fillna(method='ffill'))
aapl.count()

open              22234
close             22234
high              22234
low               22234
average           22234
volume            22258
notional          22258
numberOfTrades    22258
prev_close        22258
dtype: int64

Fill `NaN` openings with the previous valid closing price, which is interpreted as the start and end price for a minute where no activity occurred:

In [21]:
fill_cols = [ 'open', 'close', 'high', 'low', 'average', ]
for col in fill_cols:
    aapl[col].fillna(aapl.prev_close, inplace=True)
aapl.count()

open              22258
close             22258
high              22258
low               22258
average           22258
volume            22258
notional          22258
numberOfTrades    22258
prev_close        22258
dtype: int64

We expect that there are no remaining `NaN`s in the dataset (unless they appear during the first minute of the day; at the time of writing that has not been observed)

Spot check that `NaN` entries have been filled in based on the previous minute's closing price:

In [22]:
aapl[l_or(nan_idxs, nan_idxs.shift(-1).fillna(False))]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:28:00,192.990,193.000,193.050,192.990,193.014,1100,212316.000,11,192.990
2019-04-02 13:29:00,193.000,193.000,193.000,193.000,193.000,0,0.000,0,193.000
2019-04-05 13:35:00,196.535,196.540,196.540,196.535,196.537,205,40290.200,3,196.495
2019-04-05 13:36:00,196.540,196.540,196.540,196.540,196.540,0,0.000,0,196.540
2019-04-05 14:28:00,196.820,196.795,196.820,196.795,196.811,731,143868.985,13,196.820
2019-04-05 14:29:00,196.795,196.795,196.795,196.795,196.795,0,0.000,0,196.795
2019-04-15 13:32:00,198.920,198.910,198.920,198.910,198.917,400,79567.000,4,198.910
2019-04-15 13:33:00,198.910,198.910,198.910,198.910,198.910,0,0.000,0,198.910
2019-04-16 13:38:00,199.730,199.725,199.740,199.710,199.727,1104,220498.920,12,199.700


## Log-fold transforms

Market-data time-series are frequently approximated as being log-normally distributed; the fold-change from one interval to the next is considered to be normally distributed.

Here we copy the price data above, make `open` a log-fold change over the previous `close`, and express other intra-minute features as a log-fold change over their corresponding `open`ing price:

In [23]:
lg = aapl.copy()
for col in fill_cols[1:]:
    lg[col] = log(lg[col] / lg.open)
lg.open = log(lg.open / lg.prev_close)

lg.drop(columns='prev_close', inplace=True)
lg[fill_cols][l_or(nan_idxs, nan_idxs.shift(1).fillna(False))]

,open,close,high,low,average
datetime,,,,,
2019-04-02 13:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-02 13:30:00,0.000414,0.000000,0.000362,-0.000052,0.000104
2019-04-05 13:36:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-05 13:37:00,-0.000102,-0.000051,0.000000,-0.000051,-0.000010
2019-04-05 14:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-05 14:30:00,0.000076,0.000051,0.000051,-0.000102,-0.000056
2019-04-15 13:33:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-15 13:34:00,0.000050,0.000151,0.000151,-0.000025,0.000050
2019-04-16 13:39:00,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
lg.count()

open              22258
close             22258
high              22258
low               22258
average           22258
volume            22258
notional          22258
numberOfTrades    22258
dtype: int64

Normalize and rename some features:
- `trades` ⟶ `sqrt(numberOfTrades)` (variance-stabilizing transform for a Poisson distribution)
- `volume` ⟶ `avgVol` (divide out then number of trades, and `log`-transform to attempt to normalize outliers

In [25]:
lg.drop(columns=[ 'volume', 'notional', 'numberOfTrades' ], inplace=True)
lg['trades'] = sqrt(aapl.numberOfTrades)  # variance-stabilizing transform for a Poisson distribution
lg['avgVol'] = log(1 + (aapl.volume / aapl.numberOfTrades).fillna(0))  # volume-per-trade  seems extremely high-tailed, so apply log1p
lg.rename(columns={'average': 'avg', 'high': 'hi', 'low': 'lo'}, inplace=True)
lg

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735
...,...,...,...,...,...,...,...
2019-07-01 09:56:00,0.000049,-0.000197,0.000172,-0.000271,-0.000025,6.557439,4.663000
2019-07-01 09:57:00,-0.000148,0.000836,0.000959,0.000000,0.000679,4.472136,4.513055
2019-07-01 09:58:00,0.000049,0.000983,0.001449,-0.000098,0.000629,8.306624,4.540052


Add `next_close` to each minute, which is the log-fold change of the next-minute's closing price over the current minute's.

This will be treated as the "label" for a training sample anchored at each minute: given the current minute's activity (and that of several minutes prior), can we predict the next minute's closing price?

The next minute's opening- and intra-minute features (high, low, average) are not trained on / learned; IEX publishes each minute's data about halfway through to the next minute, so predicting the next minute's closing price is a good goal for the purposes of this example.

In [26]:
grouped = lg.groupby(to_date)

def get_next_close(df):
    # the next minute's closing price is a combination of:
    # - the next minute's "open" field (log-fold change over current-minute close)
    # - the next minute's "close" field (log-fold change over next-minute open)
    df['next_close'] = df.open.shift(-1) + df.close.shift(-1)
    return df
lg = grouped.apply(get_next_close)
lg

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955,0.001729
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253,-0.000262
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048,-0.000524
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817,-0.000681
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735,0.000786
...,...,...,...,...,...,...,...,...
2019-07-01 09:56:00,0.000049,-0.000197,0.000172,-0.000271,-0.000025,6.557439,4.663000,0.000689
2019-07-01 09:57:00,-0.000148,0.000836,0.000959,0.000000,0.000679,4.472136,4.513055,0.001032
2019-07-01 09:58:00,0.000049,0.000983,0.001449,-0.000098,0.000629,8.306624,4.540052,-0.000688


`next_close` was populated within each day above; that means the final minute of each day is missing a label, so we drop them:

In [27]:
lg.dropna(how='any', inplace=True)
lg.count()

open          22200
close         22200
hi            22200
lo            22200
avg           22200
trades        22200
avgVol        22200
next_close    22200
dtype: int64

## Persistent assignment of labeled samples to {training, validation} sets
Assigning samples to validation vs training should be persistent; we will train iteratively over time on the full history up to that point, and samples marked for validation should always be held out from training.

We achieve this by seeding a PRNG for each sample based on that sample's minute ID (`int(strftime('%Y%m%d%H%M'))`), and pulling a single random float. The set of samples whose floats are less than $V$ should represent $100V$% of the data, and can be used collectively as a validation-split representing proportion $V$ of the total set of labeled samples.

In [28]:
from random import random, seed
validation_split = 0.2

# decide whether a given minute should be treated as a validation (otherwise: training) test case
# seed PRNG so that this info is stable over time, and we never train on samples marked for "validation"
def get_validation_rand(dt):
    seed(int(dt.strftime('%Y%m%d%H%M')))
    return random()

datetime = lg.index.to_series()  # update index since we dropped the last minute of each day
vrs = datetime.apply(get_validation_rand).rename('validation_rand')
vfs = (vrs < validation_split).rename('validation?')
tn, vn = vfs[~vfs].count(), vfs[vfs].count()
n = tn + vn
tf, vf = tn / n, vn / n
print('%d training samples (%.2f%%), %d validation (%.2f%%)' % (tn, 100 * tf, vn, 100 * vf))

17716 training samples (79.80%), 4484 validation (20.20%)


Separate the log-transformed data into training and validation sets.

Also compute {$\mu$, $\sigma$} of the training sets, which will be used to standardize the data.

In [29]:
lg_t = lg[~vfs]
means = lg_t.mean(axis=0).rename('μ')
stddevs = lg_t.std(axis=0).rename('σ')
concat([ means, stddevs ], axis=1)

,μ,σ
open,1.203423e-05,0.000210
close,-1.029927e-05,0.000625
hi,3.176312e-04,0.000441
lo,-3.363101e-04,0.000494
avg,-2.560586e-06,0.000393
trades,4.239684e+00,1.670912
avgVol,4.522924e+00,0.329456
next_close,-3.636300e-07,0.000652


Helpers for showing descriptive statistics of each column of a DataFrame:

In [30]:
def desc(df):
    stats = DF([ describe(df[col]) for col in df.columns ]).set_index(df.columns)
    return desc_df(stats)

def desc_df(stats):
    stats['min'] = stats.minmax.apply(lambda t: t[0])
    stats['max'] = stats.minmax.apply(lambda t: t[1])
    stats['σ'] = sqrt(stats.variance)
    stats.rename(columns={ 'mean': 'μ', 'nobs': 'n' }, inplace=True)
    stats.drop(columns=[ 'minmax', 'variance' ], inplace=True)
    cs = list(stats.columns)
    stats = stats[cs[:2] + [ 'σ' ] + cs[2:-1]]
    return stats

In [31]:
desc(lg)

,n,μ,σ,skewness,kurtosis,min,max
open,22200,0.000013,0.000210,0.034395,5.150226,-0.002417,0.001974
close,22200,-0.000008,0.000618,-1.267643,51.281846,-0.018992,0.007019
hi,22200,0.000318,0.000441,3.678882,25.392481,0.000000,0.007200
lo,22200,-0.000334,0.000482,-8.720543,287.443614,-0.023854,0.000000
avg,22200,-0.000001,0.000386,-2.715888,117.037246,-0.015078,0.004867
trades,22200,4.237557,1.669046,1.157771,2.719022,0.000000,17.860571
avgVol,22200,4.522716,0.330409,-2.672563,47.591044,0.000000,7.509961
next_close,22200,0.000003,0.000647,-1.321057,42.839067,-0.018822,0.006486


The log-transform has captures the idea that, intuitively, a price is as likely to double as half. 

However, it's worth noting that the high kurtosis values imply that the data is not perfectly *normally distributed* after the log-transform; price movements are empirically "heavy-tailed", as we see here.

## Standardize data
Subtract and divide out the mean and stddev of the training features.

Note that:
- the training {$\mu$,$\sigma$} will be used for validation samples as well
- we store $-\mu/\sigma$ (pre-standardization $0$) in `zeros`, for use padding sliding windows later

In [32]:
normed = lg.copy()
zeros = -means / stddevs; zeros

for col in normed.columns:
    normed[col] = (normed[col] - means[col]) / stddevs[col]

trn = normed[~vfs]
val = normed[ vfs]
normed

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,-0.057234,-8.311713,-0.720914,-10.383990,-6.050708,1.432490,0.224707,2.652715
2019-04-01 09:31:00,1.189898,2.363861,2.609123,0.043771,0.819713,0.848142,0.322741,-0.400990
2019-04-01 09:32:00,1.934573,-1.072494,-0.364563,-2.287240,-1.444496,0.740638,-0.579366,-0.802852
2019-04-01 09:33:00,1.188069,-1.240620,-0.364414,-1.651893,-0.446198,0.572423,-0.279573,-1.044505
2019-04-01 09:34:00,-1.802120,-0.486823,-0.720914,-1.017459,-1.246735,-0.464167,-0.067349,1.206336
...,...,...,...,...,...,...,...,...
2019-07-01 09:55:00,-0.174201,-0.219630,0.506803,0.381925,0.619277,0.572423,-0.184712,-0.225791
2019-07-01 09:56:00,0.176732,-0.298372,-0.330167,0.132973,-0.056031,1.387119,0.425175,1.056565
2019-07-01 09:57:00,-0.759304,1.354359,1.456002,0.680563,1.732648,0.139117,-0.029955,1.583207


In [33]:
desc(normed)

,n,μ,σ,skewness,kurtosis,min,max
open,22200,0.002595,0.999559,0.034395,5.150226,-11.551620,9.332087
close,22200,0.002974,0.988260,-1.267643,51.281846,-30.368244,11.246940
hi,22200,0.000636,1.000447,3.678882,25.392481,-0.720914,15.621267
lo,22200,0.003692,0.974582,-8.720543,287.443614,-47.591488,0.680563
avg,22200,0.002976,0.981243,-2.715888,117.037246,-38.331684,12.382443
trades,22200,-0.001273,0.998883,1.157771,2.719022,-2.537347,8.151767
avgVol,22200,-0.000631,1.002893,-2.672563,47.591044,-13.728467,9.066579
next_close,22200,0.004523,0.992806,-1.321057,42.839067,-28.863179,9.947057


We continue to observe the presence of heavy outliers (≈10s of standard deviations, as evidenced by the `min`/`max` values in many columns). 

Dropping outliers seems unwise (sudden large price movements are arguably most important to be aware of!), so we'll proceed without further corrections.

Separate the `next_close` ("label") column, and name it `y`:

In [34]:
y = normed.next_close
normed.drop(columns='next_close', inplace=True)
normed

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,-0.057234,-8.311713,-0.720914,-10.383990,-6.050708,1.432490,0.224707
2019-04-01 09:31:00,1.189898,2.363861,2.609123,0.043771,0.819713,0.848142,0.322741
2019-04-01 09:32:00,1.934573,-1.072494,-0.364563,-2.287240,-1.444496,0.740638,-0.579366
2019-04-01 09:33:00,1.188069,-1.240620,-0.364414,-1.651893,-0.446198,0.572423,-0.279573
2019-04-01 09:34:00,-1.802120,-0.486823,-0.720914,-1.017459,-1.246735,-0.464167,-0.067349
...,...,...,...,...,...,...,...
2019-07-01 09:55:00,-0.174201,-0.219630,0.506803,0.381925,0.619277,0.572423,-0.184712
2019-07-01 09:56:00,0.176732,-0.298372,-0.330167,0.132973,-0.056031,1.387119,0.425175
2019-07-01 09:57:00,-0.759304,1.354359,1.456002,0.680563,1.732648,0.139117,-0.029955


## Rolling window, numpy `ndarray` conversion
Create a numpy array from a rolling 30-minute window of the standardized features:

In [35]:
window = 30

cols = normed.columns
x = np.moveaxis(
    array([ 
        concat(
            [ 
                normed[col].shift(i)
                for i in reversed(range(window)) 
            ], 
            axis=1
        ) \
        .fillna(zeros[col]) \
        .values
        for col in cols
    ]),
    0, 2
)
x.shape    

(22200, 30, 7)

For each minute, we have the last 30-minutes of data about the 7 features.

Partial windows are padded with "0"s (pre-standardization 0; actually $-\mu/\sigma$ in post-standardization space, taken from the `zeros` array above)

Store training/validation data, and assign the full `x` and `y` datasets as a concatenation of training and validation samples, respectively.

In [36]:
tx = x[~vfs]
vx = x[ vfs]
ty = y[~vfs].to_numpy()
vy = y[ vfs].to_numpy()
[ a.shape for a in [ tx,ty, vx,vy, x,y ] ]

[(17716, 30, 7), (17716,), (4484, 30, 7), (4484,), (22200, 30, 7), (22200,)]

In [37]:
factor = exp(y * stddevs.next_close + means.next_close).rename('factor')
next_close = (aapl.close * factor).rename('next_close')
concat([ aapl, next_close, factor ], axis=1)

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close,next_close,factor
datetime,,,,,,,,,,,
2019-04-01 09:30:00,191.645,190.65,191.645,190.600,191.189,4320,825935.940,44,191.645,190.98,1.001731
2019-04-01 09:31:00,190.700,190.98,190.980,190.640,190.761,3246,619210.510,32,190.650,190.93,0.999738
2019-04-01 09:32:00,191.060,190.93,191.090,190.780,190.951,2253,430211.740,30,190.980,190.83,0.999476
2019-04-01 09:33:00,190.980,190.83,191.010,190.760,190.946,2241,427911.290,27,190.930,190.70,0.999319
2019-04-01 09:34:00,190.760,190.70,190.760,190.600,190.666,1069,203822.465,12,190.830,190.85,1.000787
...,...,...,...,...,...,...,...,...,...,...,...
2019-07-01 09:56:00,203.280,203.24,203.315,203.225,203.275,4513,917380.920,43,203.270,203.38,1.000689
2019-07-01 09:57:00,203.210,203.38,203.405,203.210,203.348,1804,366839.520,20,203.240,203.59,1.001033
2019-07-01 09:58:00,203.390,203.59,203.685,203.370,203.518,6396,1301698.610,69,203.380,203.45,0.999312


## Build model

In [38]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [39]:
def create_model():
    model = Sequential()
    model.add(SimpleRNN(4, input_shape=(window, len(cols))))
    model.add(Dense(1))
    model.build()
    model.compile(loss='mae', optimizer='adam')
    return model

In [40]:
import re
regex = '^\d{4}-\d\d-\d\dT\d\d:\d\d$'
ckpt_dirs = sorted([
    dir
    for dir in models_dir.iterdir() 
    if re.match(regex, dir.name)
])
model = create_model()
if ckpt_dirs:
    ckpt_dir = ckpt_dirs[-1]
    print('Loading model from %s' % ckpt_dir)
    ckpt = tf.train.latest_checkpoint(str(ckpt_dir))
    print('Found model %s' % ckpt)
    model.load_weights(ckpt)
else:
    print('No pre-existing models found; using untrained model')

model.summary()

No pre-existing models found; using untrained model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 4)                 48        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 53
Trainable params: 53
Non-trainable params: 0
_________________________________________________________________


As a baseline for evaluating our model's training progress, compute the mean absolute error we'd expect from a network that simply learned to predict the mean of the training labels for every sample:

In [41]:
mean(abs(ty - mean(ty))), mean(abs(vy - mean(vy)))

(0.660055314524697, 0.6481911103412539)

In [42]:
model_ckpt_path = models_dir / run_time.strftime('%Y-%m-%dT%H:%M') / 'cp-{epoch:04d}.ckpt'; model_ckpt_path

PosixPath('/Users/ryan/c/pipelines/notebooks/models/2019-07-01T10:00/cp-{epoch:04d}.ckpt')

In [43]:
%%time
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[
              EarlyStopping(patience=100, restore_best_weights=True),
              ModelCheckpoint(str(model_ckpt_path), save_weights_only=True, save_best_only=True, period=20),
          ]
         )

Train on 17716 samples, validate on 4484 samples
Epoch 1/10000
17716/17716 [==============================] - 1s 69us/sample - loss: 0.8294 - val_loss: 0.8178
Epoch 2/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.8275 - val_loss: 0.8160
Epoch 3/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.8257 - val_loss: 0.8142
Epoch 4/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.8239 - val_loss: 0.8125
Epoch 5/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.8221 - val_loss: 0.8107
Epoch 6/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.8203 - val_loss: 0.8090
Epoch 7/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.8185 - val_loss: 0.8074
Epoch 8/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.8167 - val_loss: 0.8057
Epoch 9/10000
17716/17716 [==============================] - 0s 7us/sample - l

Epoch 74/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.7306 - val_loss: 0.7201
Epoch 75/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.7297 - val_loss: 0.7192
Epoch 76/10000
17716/17716 [==============================] - 0s 8us/sample - loss: 0.7288 - val_loss: 0.7182
Epoch 77/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.7279 - val_loss: 0.7173
Epoch 78/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.7270 - val_loss: 0.7163
Epoch 79/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.7261 - val_loss: 0.7154
Epoch 80/10000
17716/17716 [==============================] - 0s 12us/sample - loss: 0.7252 - val_loss: 0.7145
Epoch 81/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.7243 - val_loss: 0.7137
Epoch 82/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.7235 - val_loss: 0.7128
Epoch 83/

17716/17716 [==============================] - 0s 9us/sample - loss: 0.6868 - val_loss: 0.6768
Epoch 149/10000
17716/17716 [==============================] - 0s 8us/sample - loss: 0.6865 - val_loss: 0.6765
Epoch 150/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6862 - val_loss: 0.6762
Epoch 151/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6859 - val_loss: 0.6759
Epoch 152/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6856 - val_loss: 0.6756
Epoch 153/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6853 - val_loss: 0.6753
Epoch 154/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6850 - val_loss: 0.6750
Epoch 155/10000
17716/17716 [==============================] - 0s 8us/sample - loss: 0.6847 - val_loss: 0.6747
Epoch 156/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6845 - val_loss: 0.6744
Epoch 157/10000
1

Epoch 222/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6725 - val_loss: 0.6622
Epoch 223/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6724 - val_loss: 0.6621
Epoch 224/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6723 - val_loss: 0.6620
Epoch 225/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6722 - val_loss: 0.6619
Epoch 226/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6721 - val_loss: 0.6617
Epoch 227/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6720 - val_loss: 0.6616
Epoch 228/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6719 - val_loss: 0.6615
Epoch 229/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6718 - val_loss: 0.6614
Epoch 230/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6717 - val_loss: 0.6613
E

Epoch 296/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6672 - val_loss: 0.6565
Epoch 297/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6671 - val_loss: 0.6564
Epoch 298/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6671 - val_loss: 0.6564
Epoch 299/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6671 - val_loss: 0.6563
Epoch 300/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6670 - val_loss: 0.6563
Epoch 301/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6670 - val_loss: 0.6562
Epoch 302/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6669 - val_loss: 0.6562
Epoch 303/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6669 - val_loss: 0.6561
Epoch 304/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6668 - val_loss: 0.6561
E

...

17716/17716 [==============================] - 0s 8us/sample - loss: 0.6591 - val_loss: 0.6482
Epoch 1110/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6591 - val_loss: 0.6482
Epoch 1111/10000
17716/17716 [==============================] - 0s 8us/sample - loss: 0.6591 - val_loss: 0.6482
Epoch 1112/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6591 - val_loss: 0.6482
Epoch 1113/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6591 - val_loss: 0.6482
Epoch 1114/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6591 - val_loss: 0.6482
Epoch 1115/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6591 - val_loss: 0.6482
Epoch 1116/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6591 - val_loss: 0.6482
Epoch 1117/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6591 - val_loss: 0.6482
Epoch 111

Epoch 1183/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6590 - val_loss: 0.6482
Epoch 1184/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6590 - val_loss: 0.6482
Epoch 1185/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6590 - val_loss: 0.6482
Epoch 1186/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6590 - val_loss: 0.6482
Epoch 1187/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6590 - val_loss: 0.6482
Epoch 1188/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6590 - val_loss: 0.6482
Epoch 1189/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6590 - val_loss: 0.6482
Epoch 1190/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6590 - val_loss: 0.6482
Epoch 1191/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6590 - val_loss:

17716/17716 [==============================] - 0s 7us/sample - loss: 0.6589 - val_loss: 0.6481
Epoch 1257/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6589 - val_loss: 0.6481
Epoch 1258/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6589 - val_loss: 0.6481
Epoch 1259/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6589 - val_loss: 0.6481
Epoch 1260/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6589 - val_loss: 0.6481
Epoch 1261/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6589 - val_loss: 0.6481
Epoch 1262/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6589 - val_loss: 0.6481
Epoch 1263/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6589 - val_loss: 0.6481
Epoch 1264/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6589 - val_loss: 0.6481
Epoch 126

Epoch 1330/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6588 - val_loss: 0.6481
Epoch 1331/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6588 - val_loss: 0.6481
Epoch 1332/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6588 - val_loss: 0.6481
Epoch 1333/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6588 - val_loss: 0.6481
Epoch 1334/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6588 - val_loss: 0.6481
Epoch 1335/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6588 - val_loss: 0.6481
Epoch 1336/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6588 - val_loss: 0.6481
Epoch 1337/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6588 - val_loss: 0.6481
Epoch 1338/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6588 - val_loss:

17716/17716 [==============================] - 0s 7us/sample - loss: 0.6587 - val_loss: 0.6482
Epoch 1404/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6587 - val_loss: 0.6482
Epoch 1405/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6587 - val_loss: 0.6482
Epoch 1406/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6587 - val_loss: 0.6482
Epoch 1407/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6587 - val_loss: 0.6482
Epoch 1408/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6587 - val_loss: 0.6482
Epoch 1409/10000
17716/17716 [==============================] - 0s 6us/sample - loss: 0.6587 - val_loss: 0.6482
Epoch 1410/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6587 - val_loss: 0.6482
Epoch 1411/10000
17716/17716 [==============================] - 0s 7us/sample - loss: 0.6587 - val_loss: 0.6482
Epoch 141

In [44]:
def stats(model, x, y, idx, name=None, predict=True):
    if predict:
        predx = model.predict(x)[:, 0]
    else:
        predx = x

    def reindex(s):
        s = \
            idx \
            .reset_index() \
            [[ idx.index.name ]] \
            .merge(pd.Series(s).rename('x'), how='inner', left_index=True, right_index=True) \
            .set_index(idx.index.name)
        s = s[s.columns[0]]
        return s
    
    predx = reindex(predx)
    y = reindex(y)

    return stats_df(predx, y, name=name)

def stats_df(x, y, name=None):
    suffix = ' (%s)' % name if name else ''

    diffs = (y - x)
    mae = abs(diffs)
    mse = mae**2

    return DF({
        ('y^' + suffix): x,
        ('Δ' + suffix): diffs,
        ('|Δ|' + suffix): mae,
        ('Δ²' + suffix): mse,
    })

aa = aapl.merge(vfs, how='right', left_index=True, right_index=True)

def diffs_df(model):
    trn = stats(model, tx, ty, vfs[~vfs], 't')
    val = stats(model, vx, vy, vfs[ vfs], 'v')

    predx_t = trn['y^ (t)']
    predx_v = val['y^ (v)']
    predx = concat([ predx_t, predx_v ]).sort_index()
    all = stats_df(predx, y, name='*')
    
    predicted = exp(predx_t * stddevs.next_close + means.next_close) * aa[~vfs].open
    actual = aa.close.shift(-1)[~vfs]
    train_descaled = stats_df(predicted, actual, name='$').dropna(how='any')

    return concat([
        desc(trn),
        desc(val),
        desc(all),
        desc(train_descaled),
    ])

In [45]:
diffs_df(model)

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),17716,0.014681,0.064132,2.011641,15.554187,-3.052279e-01,1.139629
Δ (t),17716,-0.014681,0.998241,-1.669899,50.349329,-2.895196e+01,9.011343
|Δ| (t),17716,0.658833,0.750076,6.073539,138.107089,3.709253e-07,28.951965
Δ² (t),17716,0.996643,7.216463,91.882924,10335.791696,1.375856e-13,838.216265
y^ (v),4484,0.015886,0.063630,1.580410,8.954441,-2.736455e-01,0.584297
Δ (v),4484,0.006509,0.965281,-0.641953,13.473348,-1.229012e+01,9.774216
|Δ| (v),4484,0.648108,0.715312,3.779147,32.272513,5.074016e-05,12.290118
Δ² (v),4484,0.931601,3.662775,23.100429,783.070893,2.574564e-09,151.046993
y^ (*),22200,0.014924,0.064032,1.925907,14.248039,-3.052279e-01,1.139629
Δ (*),22200,-0.010401,0.991687,-1.479410,43.710910,-2.895196e+01,9.774216


## Predict the next closing price

In [46]:
last = x[-1]
yhat = model.predict(last.reshape((1,) + x.shape[1:]))[0][0]
factor = exp(yhat * stddevs.next_close + means.next_close)
yhat, factor

(0.13690083, 1.0000889109081643)

In [47]:
last_minute = lg.index[-1]
next_minute = last_minute + Δ(minutes=1)
last_close = aapl.loc[last_minute].close
prediction = last_close * factor
print(
    '%s: closed at %.2f\n%s: predicted close at %.2f' % (
        last_minute,
        last_close,
        next_minute,
        prediction,
    )
)

2019-07-01 09:59:00: closed at 203.45
2019-07-01 10:00:00: predicted close at 203.47


In [48]:
if next_minute in aapl.index:
    next_close = aapl.loc[next_minute].close
    print(
        'Predicted $%.2f, actually $%.2f (difference: $%.2f, %.2f%%)' % (
            prediction, 
            next_close, 
            prediction - next_close, 
            100 * (prediction - next_close) / next_close,
        )
    )

Predicted $203.47, actually $203.53 (difference: $-0.06, -0.03%)
